<h2>Rhythm patterns</h2>
Extraction of RMS amplitude as a fixed length one bar pattern,
and clustering by similarity.
Data is 698 files of ballroom dance music (1.7GB of audio).
As published in:<br />
S. Dixon, F. Gouyon and G. Widmer,
<i>Towards Characterisation of Music via Rhythmic Patterns</i>,
5th International Conference on Music Information Retrieval (ISMIR), 2004, pp 509-516.

In [ ]:
%matplotlib inline
import librosa
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans


class Obj:
    """Defines an object for storing metadata about each audio file.
    Fields are added dynamically in brdInit()."""
    name = None


def brdInit(recompute = 0):
    """Creates an array of objects, one for each file, containing path names
    for audio and annotation files, dance style, metre and tempo.
    Each audio file is in a folder defining the dance style
    (e.g. samba, cha cha, waltz). A parallel set of annotation files contain
    two integers for each piece giving the beginning and end of the first bar,
    in milliseconds.
    """
    saveFileName = 'brdFiles.npy'
    if not recompute and os.path.exists(saveFileName):
        return np.load(saveFileName, allow_pickle=True)
    dataDir='/home/simon/ballroom/beatTrack/';
    wavDir='/home/simon/ballroom/wav/';
    files = []
    for entry in os.scandir(wavDir):
        for wavFile in os.scandir(entry.path):
            obj = Obj()
            obj.baseName = dataDir + entry.name + '/' + wavFile.name[:-4]
            obj.wavFileName = wavFile.path
            obj.name = wavFile.name[:-4]
            obj.genre = entry.name
            if obj.genre[-5:] == 'Waltz':
                obj.metre = 3
            else:
                obj.metre = 4
            obj.barTimes = np.divide(np.fromfile(obj.baseName + '.txt', sep = ' '),
                                 1000)
            obj.tempo = 60 / (obj.barTimes[1] - obj.barTimes[0])
            files.append(obj)
    np.save(saveFileName, files)
    return files


def brdCreate(ln = 72, plotting = False):
    """ Finds bar-length rhythmic patterns in amplitude envelopes
    ln: samplesPerBar, must be a multiple of 24
    plotting: flag to select/deselect plots for each file
    """

    files = brdInit()
    est = [ KMeans(n_clusters = 1), KMeans(n_clusters = 2),
            KMeans(n_clusters = 3), KMeans(n_clusters = 4) ]
    if plotting:
        colours = 'rgbmcky'
        ticks = np.arange(0, 1, 0.125)  # Assume 4/4 time
        labels = [' 0 ','1/8','1/4','3/8','1/2','5/8','3/4','7/8',' 1 ']
        t = np.arange(0, 1, 1/ln)
        plt.figure(figsize = (8,5));

    for ind in range(len(files)):       # Loop over all files
        name = ('{:3d}'.format(ind) + ': ' + files[ind].genre +
                                      ': ' + files[ind].name)
        print('processing file ' + name)

        # Load and downsample audio into ln blocks per bar
        x, sr = librosa.load(files[ind].wavFileName)
        decRate = int(round(60 / files[ind].tempo * sr / ln))  # audio samples per block
        blockRate = sr / decRate                        # new sampling rate in blocks/sec
        barIndex = [int(round(bt * blockRate)) for bt in files[ind].barTimes]  # in blocks
        blockCount = int(np.floor(len(x) / decRate))
        y = np.zeros(blockCount)
        for i in range(blockCount):
            y[i] = ### CODE HERE: Calculate RMS amplitude ###

        # Align bar starts with cross-correlation
        #   - loop through bars
        #   - adjust start and end point to find sequence that correlates best with previous ones
        maxdev = round(ln*0.05)         # allow +-5% error
        barCount = 0
        barPattern = y[barIndex[0]:barIndex[0]+ln]
        while barIndex[barCount] + 2 * ln + maxdev < len(y):
            i = barIndex[barCount] + ln - 1
            barCount += 1
            offset = ### CODE HERE: calculate alignment of next bar ###
            barIndex.append(i + offset)
            barPattern = ### CODE HERE: update pattern to represent all bars seen ###
        barPattern = ### CODE HERE: normalise the global pattern ###

        # Create a 2D-array of all bar-length profiles
        allpats = np.zeros((barCount+1, ln))
        for i in range(barCount+1):
            allpats[i,:] = ### CODE HERE: select and normalise ###
        
        # Cluster bars to find patterns
        k = min(3, barCount)
        est[k].fit_predict(allpats)             # runs kmeans algorithm
        centres = est[k].cluster_centers_
        indices = est[k].labels_
        clusterSizes = ### CODE HERE ###
        bestCluster = ### CODE HERE ###
        files[ind].pattern = centres[bestCluster,:]
        files[ind].energy = barPattern
        files[ind].barIndex = barIndex
        files[ind].allPatterns = allpats

        # Display patterns
        if plotting:
            plt.cla()
            for i in range(barCount+1-1, -1, -1):
                plt.plot(t, allpats[i,:], colours[indices[i]])
            plt.xticks(ticks, labels)
            plt.plot(t, centres[bestCluster,:], color='black', linewidth=2.0)
            plt.title('Bar by bar energy patterns for ' + name)
            plt.show()
            response = input('Press enter to continue')
    
    return files


if __name__ == '__main__':
    brdCreate(plotting = True)
